In [21]:
import pandas as pd
#importing week2 modified data
df = pd.read_csv('modified_churn_data.csv', index_col='customerID', )
#removing extra columns
df = df.drop(columns=['TotalRevenue','Log_MonthlyCharges'],axis=1)
#loading new data for predection
new_data = pd.read_csv('new_churn_data.csv')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,0,0,1,29.85,29.85,0
5575-GNVDE,34,1,1,2,56.95,1889.50,0
3668-QPYBK,2,1,0,2,53.85,108.15,1
7795-CFOCW,45,0,1,3,42.30,1840.75,0
9237-HQITU,2,1,0,1,70.70,151.65,1
...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,2,84.80,1990.50,0
2234-XADUH,72,1,1,4,103.20,7362.90,0
4801-JZAZL,11,0,0,1,29.60,346.45,0


In [2]:
from pycaret.classification import *

In [3]:
# Set up PyCaret with the reduced dataset (common columns)
automl = setup(df, target='Churn')

,Description,Value
0,Session id,1789
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 7)"
4,Transformed data shape,"(7043, 7)"
5,Transformed train set shape,"(4930, 7)"
6,Transformed test set shape,"(2113, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [8]:
# Comparing models and choosing the best one based on AUC (Area Under Curve)
best_model = compare_models(sort='AUC')

# Display the best model
print("Best Model Selected:", best_model)

# Saving the best model to disk
model = load_model('best_churn_model')


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7978,0.8390,0.5047,0.6550,0.5692,0.4401,0.4471,0.1480
ada,Ada Boost Classifier,0.7935,0.8388,0.5077,0.6396,0.5656,0.4326,0.4379,0.0630
lr,Logistic Regression,0.7955,0.8376,0.5215,0.6414,0.5748,0.4422,0.4465,0.0270
lightgbm,Light Gradient Boosting Machine,0.7929,0.8268,0.5299,0.6312,0.5755,0.4400,0.4433,0.1310
qda,Quadratic Discriminant Analysis,0.7509,0.8264,0.7439,0.5219,0.6133,0.4379,0.4529,0.0120
ridge,Ridge Classifier,0.7937,0.8255,0.4526,0.6635,0.5375,0.4110,0.4237,0.0120
lda,Linear Discriminant Analysis,0.7951,0.8255,0.5077,0.6462,0.5674,0.4360,0.4421,0.0110
nb,Naive Bayes,0.7187,0.8112,0.7768,0.4820,0.5946,0.3970,0.4238,0.0130
rf,Random Forest Classifier,0.7785,0.8033,0.4939,0.6018,0.5416,0.3977,0.4015,0.1590
et,Extra Trees Classifier,0.7627,0.7812,0.4916,0.5614,0.5229,0.3662,0.3684,0.1150


Best Model Selected: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=1789, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Transformation Pipeline and Model Successfully Loaded


In [14]:
# Define the churn prediction function with inspection
def predict_churn(df, original_data):
    # Make predictions using the loaded model
    predictions = predict_model(model, data=new_data)
    # Print the columns to inspect the output
    print(predictions.columns)
        # Add the customerID to the predictions
    predictions['customerID'] = original_data['customerID']
    # Print the predictions dataframe to understand the column names
    print(predictions.head())
    # Return the churn probabilities and predicted labels with customerID
    return predictions

In [18]:
# Use the function to predict churn on the new data
predictions = predict_churn(df, new_data)

# Print the predictions
print(predictions)

Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'charge_per_tenure',
       'prediction_label', 'prediction_score'],
      dtype='object')
   customerID  tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
0  9305-CKSKC      22             1         0              2       97.400002   
1  1452-KNGVK       8             0         1              1       77.300003   
2  6723-OKKJM      28             1         0              0       28.250000   
3  7832-POPKP      62             1         0              2      101.699997   
4  6348-TACGU      10             0         0              1       51.150002   

   TotalCharges  charge_per_tenure  prediction_label  prediction_score  
0    811.700012          36.895454                 1            0.5033  
1   1701.949951         212.743744                 0            0.5016  
2    250.899994           8.960714                 0            0.5129  
3   3106.560059    

In [19]:
# saving predection file to the disk
predictions.to_csv('predictions.csv', index=False)

In [22]:
# Reading the predection file
pd = pd.read_csv('predictions.csv')
pd.head()

,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,prediction_label,prediction_score
0,9305-CKSKC,22,1,0,2,97.40,811.70,36.895454,1,0.5033
1,1452-KNGVK,8,0,1,1,77.30,1701.95,212.743740,0,0.5016
2,6723-OKKJM,28,1,0,0,28.25,250.90,8.960714,0,0.5129
3,7832-POPKP,62,1,0,2,101.70,3106.56,50.105810,1,0.5011
4,6348-TACGU,10,0,0,1,51.15,3440.97,344.097000,0,0.5038


# Summary

Using PyCaret, I applied various machine learning models to the refined churn dataset. For feature selection, I relied on statistical testing and mutual information values related to the chosen data, which yielded strong results during model comparisons. The evaluation criteria for the models focused on a balanced approach to precision and recall, given the dataset's imbalanced nature. After identifying the best-performing model, I saved it to disk for future use. I then created a Python script that loads this model to predict churn probabilities on new data. When tested with other datasets, the script successfully identified customers likely to churn, showing that those who actually did had the highest predicted probabilities. Additionally, I included a percentile ranking feature, which could be toggled on for more detailed predictions. The study's findings demonstrated strong validity for the best model, achieving an AUC of 0.84 and an accuracy of 79%.